In [ ]:
!pip install fastai==1.0.11
!python -m spacy download en
!pip install fire
from train import *

In [13]:
!rm -rf data

In [15]:
def train_grid(exp_name, sample_sizes=[0, 500, 1000, 2000, 4000, 8000, 16000, 32000, 64000, 90000]):
    for size in sample_sizes:
        for global_lm in [True, False]:
            !floyd run "bash setup.sh && python train.py mytest floyd \
                --sample-size={size} --global-lm={global_lm}" \
                --env pytorch-1.0 --gpu \
                --data frame/datasets/imdb_reviews_wt103/1:imdb_reviews_wt103 \
                -m "sample size {size}, global_lm {global_lm}"

In [16]:
train_grid('hello', sample_sizes=[0,4])

Creating project run. Total upload size: 199.2KiB
Syncing code ...
[================================] 208327/208327 - 00:00:01

JOB NAME
-------------------------------------
frame/projects/fastai_ulmfit_test/102

URL to job: https://www.floydhub.com/frame/projects/fastai_ulmfit_test/102

To view logs enter:
   floyd logs frame/projects/fastai_ulmfit_test/102
Creating project run. Total upload size: 199.2KiB
Syncing code ...
[================================] 208329/208329 - 00:00:01

JOB NAME
-------------------------------------
frame/projects/fastai_ulmfit_test/103

URL to job: https://www.floydhub.com/frame/projects/fastai_ulmfit_test/103

To view logs enter:
   floyd logs frame/projects/fastai_ulmfit_test/103
Creating project run. Total upload size: 199.2KiB
Syncing code ...
[================================] 208327/208327 - 00:00:01

JOB NAME
-------------------------------------
frame/projects/fastai_ulmfit_test/104

URL to job: https://www.floydhub.com/frame/projects/fastai_ulm